In [88]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.utils import get_column_letter

In [45]:
d1 = pd.read_csv('/Users/mac/Documents/Projects/P1/office.csv')
d2 = pd.read_csv('/Users/mac/Documents/Projects/P1/bank.csv')

In [46]:
d1 = d1.sort_values(by='A/c').reset_index(drop=True)
d1['Sno'] = range(1,len(d1)+1)
d1.head()

,Sno,Name,A/c,ST,OA/c
0,1,AMARJIT SINGH S/O MANN SINGH,23400103100008,95260.0,1545
1,2,LAKHVIR SINGH S/O BHOLA SINGH,23400103100009,19400.0,1546
2,3,JAGJIT SINGH S/O PAL SINGH,23400103100010,44650.0,1547
3,4,MALKIT KAUR W/O MOHINDER SINGH,23400103100011,35075.0,1548
4,5,BINDER SINGH S/O BALVEER SINGH,23400103100012,52000.0,1549


In [47]:
d2.sort_values(by='A/c').reset_index(drop=True)
d2['Sno'] = range(1,len(d2)+1)

In [61]:
d_common = d1.merge(d2, on="A/c", suffixes=('_d1', '_d2'))
d_not_common = d1.merge(d2, on="A/c", how="outer", indicator=True)
# d_common['Sno_d1'] = range(1,len('Sno_d1')+1)
# d_common['Sno_d2'] = range(1,len('Sno_d2')+1)


In [62]:
only_in_d1 = d_not_common[d_not_common['_merge'] == 'left_only'].drop(columns=['_merge'])
only_in_d2 = d_not_common[d_not_common['_merge'] == 'right_only'].drop(columns=['_merge'])


In [63]:
d_common.head()

,Sno_d1,Name_d1,A/c,ST_d1,OA/c,Sno_d2,Name_d2,ST_d2
0,1,AMARJIT SINGH S/O MANN SINGH,23400103100008,95260.0,1545,1,AMARJIT SINGH S/O MANN S,-95260.0
1,2,LAKHVIR SINGH S/O BHOLA SINGH,23400103100009,19400.0,1546,2,LAKHVEER SINGH S/O BHOLA S,-19400.0
2,3,JAGJIT SINGH S/O PAL SINGH,23400103100010,44650.0,1547,3,GURTEJ URF JAGJIT S/O PAL S,-44650.0
3,4,MALKIT KAUR W/O MOHINDER SINGH,23400103100011,35075.0,1548,4,MALKIT KAUR W/O MOHINDER S,-35075.0
4,5,BINDER SINGH S/O BALVEER SINGH,23400103100012,52000.0,1549,5,BINDER SINGH S/O BALVEER S,-52000.0


In [64]:
d_common["Difference"] = d_common["ST_d1"] + d_common["ST_d2"]
d_common = d_common[d_common['Difference']!=0]
#d_common['Sno'] = range(1,len(d1)+1)

In [65]:
d_common.head()

,Sno_d1,Name_d1,A/c,ST_d1,OA/c,Sno_d2,Name_d2,ST_d2,Difference
17,18,PAPU SINGH S/O JANGIR SINGH,23400103100134,6995.0,1564,18,PAPPU SINGH S/O JANGIR SINGH,-26000.0,-19005.0
29,30,SUKHWINDER KAUR S/O GURJANT SINGH,23400103100314,79645.0,1599,31,SUKHWINDER KAUR W/O GURJANT,-67030.0,12615.0
30,31,DEEPAK KUMAR S/O RULDU RAM,23400103100315,8725.0,1618,32,"DEEPAK KUMAR S/O RULDU RAM,",-34725.0,-26000.0
33,34,BUTA SINGH S/O SUKHDEV SINGH,23400103100318,88600.0,1579,35,BOOTA SINGH S/O SUKHDEV SINGH,-79600.0,9000.0
34,35,SUKHWINDER SINGH S/O SUKHDEV SINGH,23400103100319,80855.0,1578,36,SUKHWINDER SINGH S/O SUKHDEV,-75930.0,4925.0


In [66]:
d_common.to_csv("common_accounts.csv", index=False)
only_in_d1.to_csv("only_in_d1.csv", index=False)
only_in_d2.to_csv("only_in_d2.csv", index=False)

In [87]:
d_common.to_excel("common.xlsx", index=False)

In [93]:
d_common.to_excel('/Users/mac/Documents/Projects/P1/common.xlsx', index=False, engine='openpyxl')
wb = load_workbook('/Users/mac/Documents/Projects/P1/common.xlsx')
ws = wb.active
orange_fill = PatternFill(start_color="FABF8F", end_color="FABF8F", fill_type="solid")  # Orange (Columns 1-5)
blue_fill = PatternFill(start_color="92CDDC", end_color="92CDDC", fill_type="solid")  # Blue (Columns 6-8)
green_fill = PatternFill(start_color="C4D79B", end_color="C4D79B", fill_type="solid")  # Green (Column 9)

for col_idx, column in enumerate(d_common.columns, start=1):
    cell = ws[f"{get_column_letter(col_idx)}1"]
    
    cell.font = Font(bold=True, color="000000", size=20)  # Bold + White Text + Bigger Size
    cell.alignment = Alignment(horizontal="center")  # Centered Text
    # Apply color based on column position
    if 1 <= col_idx <= 5:
        cell.fill = orange_fill
    elif 6 <= col_idx <= 8:
        cell.fill = blue_fill
    elif col_idx == 9:
        cell.fill = green_fill

# Auto Adjust Column Widths
for col_idx, column in enumerate(d_common.columns, start=1):
    max_length = max(len(str(val)) for val in d_common[column]) + 2
    ws.column_dimensions[get_column_letter(col_idx)].width = max_length

wb.save('/Users/mac/Documents/Projects/P1/common.xlsx')